In [ ]:
HF_KEY = 'hsdfsddsfgdsf'  # replace with your own token: huggingface.co/settings/tokens

# Grabbing a dataset from the repository

In [3]:
from datasets import load_dataset

# https://huggingface.co/datasets/urvog/llama2_transcripts_healthcare_callcenter
callcenter_dataset = load_dataset("urvog/llama2_transcripts_healthcare_callcenter")

callcenter_dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 1000
    })
})

In [31]:
type(callcenter_dataset)

datasets.dataset_dict.DatasetDict

In [4]:
callcenter_dataset['train'][0]

{'text': "<s>[INST] Classify the following call transcript:\n\nAgent 3: Thank you for calling HealthHarbor, my name is Agent 3. How can I assist you today?\n\nCustomer: Hi Agent 3, my name is Emma Johnson. I've been experiencing some symptoms lately and I wanted to seek medical advice or get a symptom assessment.\n\nAgent 3: I'm sorry to hear that, Emma. I'll do my best to help you. Can you please describe the symptoms you've been experiencing?\n\nCustomer: Sure. I've been having a persistent headache for the past few days, and it's been accompanied by dizziness and occasional nausea. I'm not sure what could be causing it.\n\nAgent 3: I understand your concern, Emma. Headaches can have various causes. Have you experienced any recent changes in your lifestyle or any other symptoms besides the headache, dizziness, and nausea?\n\nCustomer: No major lifestyle changes, but I have noticed that my vision seems a bit blurry at times. And I've been feeling more fatigued than usual.\n\nAgent 3: 

In [33]:
print(callcenter_dataset['train'][0]['text'])

<s>[INST] Classify the following call transcript:

Agent 3: Thank you for calling HealthHarbor, my name is Agent 3. How can I assist you today?

Customer: Hi Agent 3, my name is Emma Johnson. I've been experiencing some symptoms lately and I wanted to seek medical advice or get a symptom assessment.

Agent 3: I'm sorry to hear that, Emma. I'll do my best to help you. Can you please describe the symptoms you've been experiencing?

Customer: Sure. I've been having a persistent headache for the past few days, and it's been accompanied by dizziness and occasional nausea. I'm not sure what could be causing it.

Agent 3: I understand your concern, Emma. Headaches can have various causes. Have you experienced any recent changes in your lifestyle or any other symptoms besides the headache, dizziness, and nausea?

Customer: No major lifestyle changes, but I have noticed that my vision seems a bit blurry at times. And I've been feeling more fatigued than usual.

Agent 3: Thank you for sharing th

# Grabbing a model from the repository

In [5]:
from transformers import AutoModel, AutoTokenizer  # the auto classes automatically get the right python classes

In [6]:
MODEL_ID = 'bert-base-uncased'

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)  # is an instance of BertTokenizerFast

tokenizer

BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True)

In [7]:

model = AutoModel.from_pretrained(MODEL_ID)  # is an instance of  BertModel

model

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

# Text classification(positive or negative)

In [8]:
from transformers import pipeline  # the pipeline object lets us use fine-tuned models off the shelf

pipe = pipeline("text-classification", model="ProsusAI/finbert")

In [9]:
pipe(
    [
        'Stocks rallied as the dollar grew in strength',
        '$AAPL down heavily in the after hours'
    ]
)

[{'label': 'positive', 'score': 0.8791128396987915},
 {'label': 'negative', 'score': 0.9381498098373413}]

# Using the Inference Endpoint (this one is a bert model fine-tuned to display if a commentis toxic or not)

In [13]:
import requests

API_URL = "https://d2q5h5r3a1pkorfp.us-east-1.aws.endpoints.huggingface.cloud"
headers = {
	"Accept" : "application/json",
	"Authorization": f"Bearer {HF_KEY}",
	"Content-Type": "application/json" 
}

def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()

output = query({
    "inputs": ['You are such a loser'],
    "parameters": {}
})
output

[{'label': 'Toxic', 'score': 0.6511591076850891}]

In [10]:
#using the same model locally. NO data is passed to huggingface or aws

toxic_pipeline = pipeline(
    'text-classification',
    model='profoz/distilbert-toxic-classifier'
)


config.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

C:\Users\parth\anaconda3\lib\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\parth\.cache\huggingface\hub\models--profoz--distilbert-toxic-classifier. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/392 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [11]:
toxic_pipeline(
    [
        "You are such a loser"
    ]
)

[{'label': 'Toxic', 'score': 0.6511591672897339}]

In [27]:
import requests

API_URL = "https://api-inference.huggingface.co/models/profoz/distilbert-toxic-classifier"
headers = {
	"Accept" : "application/json",
	"Authorization": f"Bearer {HF_KEY}",
	"Content-Type": "application/json" 
}


In [28]:
def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()


In [29]:
output = query({
    "inputs": ['You are such a loser'],
    "parameters": {}
})

In [30]:
output

[[{'label': 'Toxic', 'score': 0.6511591076850891},
  {'label': 'Non-Toxic', 'score': 0.3488408625125885}]]

https://api-inference.huggingface.co/models/tensor-trek/distilbert-toxicity-classifier

In [14]:
import requests

API_URL = "https://api-inference.huggingface.co/models/tensor-trek/distilbert-toxicity-classifier"
headers = {
	"Accept" : "application/json",
	"Authorization": f"Bearer {HF_KEY}",
	"Content-Type": "application/json" 
}


In [15]:
def query(payload):
	response = requests.post(API_URL, headers=headers, json=payload)
	return response.json()


In [16]:
output = query({
    "inputs": ['You are such a loser'],
    "parameters": {}
})

In [17]:
output

[[{'label': 'TOXIC', 'score': 0.9990478157997131},
  {'label': 'NEUTRAL', 'score': 0.0009522190084680915}]]